# 7. Stats
Source: <br> 
1. FanGraphs API <br>

This imports stats from Fangraphs <br>
This calculates stats that aren't used in the models but help us get there <br>

### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import glob
import warnings
warnings.filterwarnings("ignore")
import datetime
from datetime import date
import time
import re
import pickle

import sys
sys.path.append(r"C:\Users\james\Documents\MLB\Code")
from Utilities import *

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

model_path = r"C:\Users\james\Documents\MLB\Code\Models"
baseball_path = r"C:\Users\james\Documents\MLB\Data2"
download_path = r"C:\Users\james\Downloads"

In [2]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")


In [3]:
 # This reads in Chadwick register with player codes.
keep_list = ['key_mlbam', 'key_fangraphs', 'name_first', 'name_last']
chadwick = read_chadwick(keep_list)

In [4]:
crosswalk = pd.read_csv(os.path.join(baseball_path, "Utilities", "Crosswalk.csv"), encoding='iso-8859-1') 
crosswalk = crosswalk[['mlbamid', 'steamerid']]
crosswalk.rename(columns={'mlbamid':'mlbamid_fill'}, inplace=True)

crosswalk.drop_duplicates('steamerid', keep='last', inplace=True)

### Scrape FanGraphs

In [5]:
# Fangraphs API
def scrape_batters():
    # Read in API json
    batters_lb = pd.read_json('https://www.fangraphs.com/api/projections?type=steamer&stats=bat&pos=all&team=0&players=0&lg=all')
    
    # Name is currently some weird thing with a bunch of data. We don't need all that
    batters_lb['Name'] = batters_lb['PlayerName']
    # Rename to match steam. Note that steamerid = key_fangraphs
    batters_lb.rename(columns={'playerids':'steamerid'}, inplace=True)
    # Convert to string
    chadwick['key_fangraphs'] = (chadwick['key_fangraphs']).astype('str')
    # Remove trailing .0
    chadwick['key_fangraphs'] = chadwick['key_fangraphs'].str.replace(r'\.\d', "", regex=True)
    
    # Merge with chadwick for mlbamid
    batters_lb = batters_lb.merge(chadwick, left_on='steamerid', right_on='key_fangraphs', how='left')
    batters_lb = batters_lb.merge(crosswalk, on='steamerid', how='left')
    
    
    # Create missing columns to match what's provided by steamer 
    batters_lb['proj_date'] = todaysdate
    batters_lb['mlbamid'] = batters_lb['key_mlbam']
    batters_lb['bats'] = "MI" # Not included in FanGraphs data
    batters_lb['playerid'] = batters_lb['steamerid']
    batters_lb['NIBB'] = batters_lb['BB'] - batters_lb['IBB']
    batters_lb.rename(columns={'name_first':'firstname', 'name_last':'lastname', 'minpos':'position'}, inplace=True)

    batters_lb['mlbamid'].fillna(batters_lb['mlbamid_fill'], inplace=True)
    
    # Keep relevant variables and in order
    batters_lb = batters_lb[['proj_date', 'mlbamid', 'steamerid', 'firstname', 'lastname', 'Team', 'position', 'bats', 
                             'PA', 'IBB', 'NIBB', 'BB', 'SO', 'HBP', 'H', '2B', '3B', 'HR', 'OBP', 'SLG', 'wOBA', 'SB', 'CS', 'playerid', 'Name']]


    print(batters_lb[batters_lb['Name'] == "Jordan Walker"])    

    
    # Export to CSV
    batters_lb.to_csv(os.path.join(baseball_path, "7. Stats", "A. Raw FanGraphs", "Batters", "Batters_FG_" + todaysdate + ".csv"), encoding='iso-8859-1')
    
scrape_batters()

    proj_date   mlbamid steamerid firstname lastname Team position bats   PA  \
279  20230707  691023.0     27475    Jordan   Walker  STL       OF   MI  514   

     IBB  ...  2B  3B  HR       OBP       SLG     wOBA  SB  CS  playerid  \
279    1  ...  25   2  13  0.305773  0.393533  0.30609   8   4     27475   

              Name  
279  Jordan Walker  

[1 rows x 25 columns]


In [6]:
# Fangraphs API
def scrape_pitchers():
    # Read in API json
    pitchers_lb = pd.read_json('https://www.fangraphs.com/api/projections?type=steamer&stats=pit&pos=all&team=0&players=0&lg=all')

    # Name is currently some weird thing with a bunch of data. We don't need all that
    pitchers_lb['Name'] = pitchers_lb['PlayerName']
    # Rename to match steam. Note that steamerid = key_fangraphs
    pitchers_lb.rename(columns={'playerids':'steamerid'}, inplace=True)
    # Convert to string
    chadwick['key_fangraphs'] = (chadwick['key_fangraphs']).astype('str')
    # Remove trailing .0
    chadwick['key_fangraphs'] = chadwick['key_fangraphs'].str.replace(r'\.\d', "", regex=True)
    
    # Merge with chadwick for mlbamid
    pitchers_lb = pitchers_lb.merge(chadwick, left_on='steamerid', right_on='key_fangraphs', how='left')
    pitchers_lb = pitchers_lb.merge(crosswalk, on='steamerid', how='left')


    # Create missing columns to match what's provided by steamer 
    pitchers_lb['proj_date'] = todaysdate
    pitchers_lb['mlbamid'] = pitchers_lb['key_mlbam']
    pitchers_lb['Throws'] = "MI" # Not included in FanGraphs data
    pitchers_lb['playerid'] = pitchers_lb['steamerid']
    pitchers_lb.rename(columns={'name_first':'firstname', 'name_last':'lastname', 'minpos':'position'}, inplace=True)

    pitchers_lb['mlbamid'].fillna(pitchers_lb['mlbamid_fill'], inplace=True)

    
    # Keep relevant variables and in order
    pitchers_lb = pitchers_lb[['proj_date', 'mlbamid', 'steamerid', 'firstname', 'lastname', 'Throws', 
                               'IP', 'G', 'GS', 'K/9', 'BB/9', 'H', 'HR', 'playerid', 'Name']]
    
    # Export to CSV
    pitchers_lb.to_csv(os.path.join(baseball_path, "7. Stats", "A. Raw FanGraphs", "Pitchers", "Pitchers_FG_" + todaysdate + ".csv"), encoding='iso-8859-1')
    
scrape_pitchers()

### Create Useful Stats

In [7]:
def create_intermediate_batters(date):
    # Read in file
    filename = "Batters_FG_" + date + ".csv"
    df = pd.read_csv(os.path.join(baseball_path, "7. Stats", "A. Raw FanGraphs", "Batters", filename), encoding='iso-8859-1')
    # Create singles
    df['1B'] = df['H'] - df['2B'] - df['3B'] - df['HR']
    
    # Basic stats
    hit_list = ['1B', '2B', '3B', 'HR', 'BB', 'HBP', 'SO']

    # Advance stats
    rate_list = ['OBP', 'SLG', 'wOBA']
    for stat in hit_list:
        rate = stat + "_rate"
        rate_list.append(rate)
        df[rate] = df[stat] / df['PA']

    df['SBA'] = df['SB'] + df['CS']
    df['SBO'] = df['1B'] + df['BB'] + df['HBP']
    df['sba_imp'] = df['SBA'] / df['SBO']

    # Cap imputed SBA 
    df['sba_imp'] = np.where(df['sba_imp'] > 0.15, 0.15, df['sba_imp'])

    # Determine stolen base success rate
    df['sbr'] = df['SB'] / df['SBA']
    
    keep_list = ['Name', 'mlbamid', 'playerid', 'sba_imp', 'sbr'] + rate_list
    df = df[keep_list]
    
    df['sbr'].fillna(0.6, inplace=True) # assume 25th percentile 
    df['sba_imp'].fillna(0.05, inplace=True) # assume low prob
    
    df.columns = df.columns.str.lower()
    df.rename(columns={'1b_rate': 'b1_rate', '2b_rate': 'b2_rate', '3b_rate': 'b3_rate'}, inplace=True)

    df.dropna(inplace=True)
    
    sba_2b_reg = pickle.load(open(os.path.join(model_path, 'sba_2b_20220901.sav'), 'rb'))
    df['sba_2b'] = sba_2b_reg.predict(df[['sba_imp']])

    sba_3b_reg = pickle.load(open(os.path.join(model_path, 'sba_3b_20220901.sav'), 'rb'))
    df['sba_3b'] = sba_3b_reg.predict(df[['sba_imp']])

    sb_2b_reg = pickle.load(open(os.path.join(model_path, 'sb_2b_20220901.sav'), 'rb'))
    df['sb_2b'] = sb_2b_reg.predict(df[['sbr']])

    sb_3b_reg = pickle.load(open(os.path.join(model_path, 'sb_3b_20220901.sav'), 'rb'))
    df['sb_3b'] = sb_3b_reg.predict(df[['sbr']])
       
    # Merge with chadwick to fix certain fangraphs ids 
    df = df.merge(chadwick, left_on='mlbamid', right_on='key_mlbam', how='left')
    # df['key_fangraphs'] = df['key_fangraphs'].astype('int', errors='ignore')
    df['playerid'] = np.where(df['playerid'].str.startswith("sa") & ~df['key_fangraphs'].isna(), df['key_fangraphs'], df['playerid'])
    df['playerid'] = df['playerid'].astype('string')
    df['playerid'] = df['playerid'].str.replace(r'\.0', '', regex=True)
        
    df.drop(columns={'index', 'key_fangraphs', 'key_mlbam', 'name_first', 'name_last'}, inplace=True)
    
    # Write intermediate FanGraphs data to csv
    df.to_csv(os.path.join(baseball_path, "7. Stats", "B. Clean FanGraphs", "Batters", "Batters_FG2_" + date + ".csv"), encoding='iso-8859-1')
        
    return df

In [8]:
def create_intermediate_pitchers(date):
    # Read in file
    filename = "Pitchers_FG_" + date + ".csv"
    df = pd.read_csv(os.path.join(baseball_path, "7. Stats", "A. Raw FanGraphs", "Pitchers", filename), encoding='iso-8859-1')
    
    df['H9'] = df['H'] / df['IP'] * 9
    df['HR9'] = df['HR'] / df['IP'] * 9
    
    df.rename(columns={'K/9':'K9', 'BB/9':'BB9'}, inplace=True)
    
    keep_list = ['playerid', 'mlbamid', 'H9', 'HR9', 'K9', 'BB9'] 
    df = df[keep_list]
    
    # Merge with chadwick to fix certain fangraphs ids 
    df = df.merge(chadwick, left_on='mlbamid', right_on='key_mlbam', how='left')
    # df['key_fangraphs'] = df['key_fangraphs'].astype('int', errors='ignore')
    df['playerid'] = np.where(df['playerid'].str.startswith("sa") & ~df['key_fangraphs'].isna(), df['key_fangraphs'], df['playerid'])
    df['playerid'] = df['playerid'].astype('string')
    df['playerid'] = df['playerid'].str.replace(r'\.0', '', regex=True)
    
    df.drop(columns={'index', 'key_fangraphs', 'key_mlbam', 'name_first', 'name_last'}, inplace=True)
    
    
    # Write intermediate FanGraphs data to csv
    df.to_csv(os.path.join(baseball_path, "7. Stats", "B. Clean FanGraphs", "Pitchers", "Pitchers_FG2_" + date + ".csv"), encoding='iso-8859-1')
    
    return df

In [9]:
def batter_merge(date):
    # Read in batter stats from API
    batter_filename = "Batters" + date + ".csv"
    batters_api = pd.read_csv(os.path.join(baseball_path, "4. Dataset", "Batters", batter_filename), encoding='iso-8859-1')
    
    # Make string, remove trailing .0
    batters_api = fix_fangraphs(batters_api)
    
    # Read in batter projections from FanGraphs
    batters_fg = create_intermediate_batters(date)
    batters_fg['key_fangraphs'] = batters_fg['playerid']
    
    
    # Merge API data with FG data
    batters_df = batters_api.merge(batters_fg, left_on='key_mlbam', right_on='mlbamid', how='outer')
    
    return batters_df

In [10]:
def pitcher_merge(date):
    # Read in pitcher stats from API
    pitcher_filename = "Pitchers" + date + ".csv"
    pitchers_api = pd.read_csv(os.path.join(baseball_path, "4. Dataset", "Pitchers", pitcher_filename), encoding='iso-8859-1')
    
    # Make string, remove trailing .0
    pitchers_api = fix_fangraphs(pitchers_api)
    
    # Read in pitcher projections from FanGraphs
    pitchers_fg = create_intermediate_pitchers(date)
    pitchers_fg['key_fangraphs'] = pitchers_fg['playerid']

    # Merge API data with FG data
    pitchers_df = pitchers_api.merge(pitchers_fg, left_on='key_mlbam', right_on='mlbamid', how='outer')

    
    return pitchers_df

In [11]:
os.chdir(r"C:\Users\james\Documents\MLB\Code\Models")

kmeans_model_filename = "model_batter_cluster.pkl"
with open(kmeans_model_filename, "rb") as file:
    batter_kmeans = pickle.load(file)
    
batter_stats_fg = ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate', 'woba', 'slg', 'obp']

kmeans_model_filename = "model_pitcher_cluster.pkl"
with open(kmeans_model_filename, "rb") as file:
    pitcher_kmeans = pickle.load(file)
    
pitcher_stats_fg = ['H9','HR9','K9','BB9']

In [12]:
## Normal stats  
# Standardize the data using StandardScaler
scaler_filename = "batter_stats_scaler.pkl"
with open(scaler_filename, "rb") as file:
    batter_scaler = pickle.load(file)
    
scaler_filename = "pitcher_stats_scaler.pkl"
with open(scaler_filename, "rb") as file:
    pitcher_scaler = pickle.load(file)

In [13]:
## FG Stats
# Standardize the data using StandardScaler
scaler_filename = "batter_stats_fg_scaler.pkl"
with open(scaler_filename, "rb") as file:
    batter_fg_scaler = pickle.load(file)
    
scaler_filename = "pitcher_stats_fg_scaler.pkl"
with open(scaler_filename, "rb") as file:
    pitcher_fg_scaler = pickle.load(file)

In [14]:
batter_clusters = pd.read_csv(os.path.join(baseball_path, "Inputs", "Batter Clusters.csv"))
pitcher_clusters = pd.read_csv(os.path.join(baseball_path, "Inputs", "Pitcher Clusters.csv"))# Inputs
batter_stats_short = ['b1_b','b2_b','b3_b','hr_b','bb_b','hbp_b',
                'so_b','fo_b','go_b','lo_b','po_b',
                'iso_b','slg_b','obp_b','woba_b',
                'to_left_b','to_middle_b','to_right_b',
                'hard_hit_b','totalDistance_b','launchSpeed_b','maxSpeed_b','maxSpin_b',
                'ab_b','pa_b']

batter_stats_long =  ['b1_b_long','b2_b_long','b3_b_long','hr_b_long','bb_b_long','hbp_b_long',
                'so_b_long','fo_b_long','go_b_long','lo_b_long','po_b_long',
                'iso_b_long','slg_b_long','obp_b_long','woba_b_long',
                'to_left_b_long','to_middle_b_long','to_right_b_long',
                'hard_hit_b_long','totalDistance_b_long','launchSpeed_b_long','maxSpeed_b_long','maxSpin_b_long',
                'ab_b_long','pa_b_long']

pitcher_stats_short = ['b1_p','b2_p','b3_p','hr_p','bb_p','hbp_p',
                 'so_p','fo_p','go_p','lo_p','po_p',
                 'iso_p','slg_p','obp_p','woba_p',
                 'to_left_p','to_middle_p','to_right_p',
                 'hard_hit_p','totalDistance_p','launchSpeed_p','maxSpeed_p','maxSpin_p',
                 'ab_p','pa_p']

pitcher_stats_long = ['b1_p_long','b2_p_long','b3_p_long','hr_p_long','bb_p_long','hbp_p_long',
                 'so_p_long','fo_p_long','go_p_long','lo_p_long','po_p_long',
                 'iso_p_long','slg_p_long','obp_p_long','woba_p_long',
                 'to_left_p_long','to_middle_p_long','to_right_p_long',
                 'hard_hit_p_long','totalDistance_p_long','launchSpeed_p_long','maxSpeed_p_long','maxSpin_p_long',
                 'ab_p_long','pa_p_long']

batter_stats_fg = ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate', 'woba', 'slg', 'obp']

pitcher_stats_fg = ['H/9','HR/9','K/9','BB/9']


venues = ['venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5', 'venue_7', 'venue_10', 'venue_12', 
          'venue_13', 'venue_14', 'venue_15', 'venue_16', 'venue_17', 'venue_19', 'venue_22', 'venue_31', 
          'venue_32', 'venue_680', 'venue_2392', 'venue_2394', 'venue_2395', 'venue_2535', 'venue_2536', 
          'venue_2602', 'venue_2680', 'venue_2681', 'venue_2701', 'venue_2735', 'venue_2756', 'venue_2889', 
          'venue_3289', 'venue_3309', 'venue_3312', 'venue_3313', 'venue_4169', 'venue_4705', 'venue_5010', 
          'venue_5325', 'venue_5365', 'venue_5381', 'venue_5445']

years = ['year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']

other_list = ['p_L','b_L','x_vect','y_vect','temperature','onFirst','onSecond','onThird','inning','top','score_diff']

In [15]:
# Remove items that do not apply to batters or pitchers
batter_stats_short.remove("maxSpeed_b")
batter_stats_short.remove("maxSpin_b")
batter_stats_long.remove("maxSpeed_b_long")
batter_stats_long.remove("maxSpin_b_long")

pitcher_stats_short.remove("totalDistance_p")
pitcher_stats_long.remove("totalDistance_p_long")
pitcher_stats_short.remove("launchSpeed_p")
pitcher_stats_long.remove("launchSpeed_p_long")


# # Not sure why but these are weird
# batter_stats.remove("launchSpeed_b")
# batter_stats.remove("totalDistance_b")
# batter_stats.remove("launchSpeed_b_long")
# batter_stats.remove("totalDistance_b_long")

# pitcher_stats.remove("maxSpeed_p")
# pitcher_stats.remove("maxSpin_p")
# pitcher_stats.remove("maxSpeed_p_long")
# pitcher_stats.remove("maxSpin_p_long")


# Testing removing these
batter_stats_short.remove('ab_b')
batter_stats_short.remove('pa_b')
batter_stats_long.remove('ab_b_long')
batter_stats_long.remove('pa_b_long')

pitcher_stats_short.remove('ab_p')
pitcher_stats_short.remove('pa_p')
pitcher_stats_long.remove('ab_p_long')
pitcher_stats_long.remove('pa_p_long')

batter_stats = batter_stats_short + batter_stats_long
pitcher_stats = pitcher_stats_short + pitcher_stats_long

### Create Rosters

In [16]:
def create_team_rosters(date=todaysdate):
    # Create new folder with daily rosters
    team_folder = "Daily" + date
    try:
        os.mkdir(os.path.join(baseball_path, "7. Stats", "C. Teams", team_folder))
    except:
        pass
    
    # Locate daily rosters
    rosters_folder = "Rosters" + date
    rosters_path = os.path.join(baseball_path, "6. Rosters", rosters_folder)
    
    
    # Merge API and FG data
    batters_df = batter_merge(date)
    pitchers_df = pitcher_merge(date)    

    
    for filename in os.listdir(rosters_path):
        print(filename)
        # Read in roster
        df = pd.read_csv(os.path.join(rosters_path, filename), encoding='iso-8859-1')

        # Destination     
        excel_file = filename.replace(".csv", "")
        excel_file = excel_file + ".xlsx"
        file_name = os.path.join(baseball_path, "7. Stats", "C. Teams", team_folder, excel_file)


        ### Batters
        batters_merged = df.merge(batters_df, left_on='id', right_on='batter', how='left', suffixes=("", "_api"))
        
        # Only keep batters
        batters_merged = batters_merged.query('position != "P"')
        
        # Create dummy variable for if they're a lefty. This is necessary to project. (maybe move this)
        batters_merged['b_L'] = np.where(batters_merged['batSide'] == "Left", 1, 0)
        
        
        # Standardize data 
        batter_stats_fg = ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate', 'woba', 'slg', 'obp']
        batters_merged[batter_stats_fg] = batter_fg_scaler.transform(batters_merged[batter_stats_fg])

        for stat in batter_stats_fg:
            batters_merged[stat].fillna(batters_merged[stat].mean(), inplace=True)
        
        # Determine cluster
        batters_merged['Cluster'] = np.nan
        for i, row in batters_merged.iterrows():
            try:
                prediction = batter_kmeans.predict([row[['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate', 'woba', 'slg', 'obp', 'b_L']]])
                # print(f"Prediction: {prediction}")
                batters_merged.loc[i, 'Cluster'] = prediction[0]
            except KeyError as err:
                print(f"KeyError: {err}")
                batters_merged.loc[i, 'Cluster'] = 1
                
        # Merge with clusters
        batters_merged = pd.merge(batters_merged, batter_clusters, on='Cluster', suffixes=("", "_cl"), how='left')
        
        # Fill in small sample 
        for stat in batter_stats:
            batters_merged[f'{stat}_l'] = np.where(batters_merged['pa_b_l'] < 40, batters_merged[f'{stat}_l_cl'], batters_merged[f'{stat}_l'])
            batters_merged[f'{stat}_r'] = np.where(batters_merged['pa_b_r'] < 40, batters_merged[f'{stat}_r_cl'], batters_merged[f'{stat}_r'])
            
        # Drop cluster columns
        columns_to_drop = [col for col in batters_merged.columns if col.endswith('_cl')]
        batters_merged.drop(columns=columns_to_drop, inplace=True)
            
            
        ## Normalize stats
        # Left
        # Rename stats to make compatible with scaler
        for stat in batter_stats:
            # Get the column name with the '_l' suffix
            column_l = stat + '_l'

            # Rename the column by removing the '_l' suffix
            batters_merged.rename(columns={column_l: stat}, inplace=True)

        # Apply the scaler to the renamed column
        batters_merged[batter_stats] = batter_scaler.transform(batters_merged[batter_stats])

        for stat in batter_stats:
            # Get the column name with the '_l' suffix
            column_l = stat + '_l'
            
            batters_merged.rename(columns={stat: column_l}, inplace=True)
            
        # Right
        for stat in batter_stats:
            # Get the column name with the '_r' suffix
            column_r = stat + '_r'

            # Rename the column by removing the '_r' suffix
            batters_merged.rename(columns={column_r: stat}, inplace=True)

        # Apply the scaler to the renamed column
        batters_merged[batter_stats] = batter_scaler.transform(batters_merged[batter_stats])

        for stat in batter_stats:
            # Get the column name with the '_r' suffix
            column_r = stat + '_r'
            
            batters_merged.rename(columns={stat: column_r}, inplace=True)
        
        
        # Save as Excel
        batters_merged.to_excel(file_name, sheet_name="Batters", engine='openpyxl')
        

        ### Pitcher
        pitchers_merged = df.merge(pitchers_df, left_on='id', right_on='pitcher', how='left', suffixes=("", "_api"))
        
        # Only keep pitchers
        desired_positions = ['P', 'TWP', 'Pitcher', 'Two-Way Player']
        pitchers_merged = pitchers_merged[pitchers_merged['position'].isin(desired_positions)]


        # Create dummy variable for if they're a lefty. This is necessary to project. (maybe move this)
        pitchers_merged['p_L'] = np.where(pitchers_merged['pitchHand'] == "L", 1, 0)
        
        pitchers_merged.rename(columns={'H9':'H/9', 'HR9':'HR/9','K9':'K/9','BB9':'BB/9'}, inplace=True)
        pitcher_stats_fg = ['H/9', 'HR/9','K/9', 'BB/9']
        pitchers_merged[pitcher_stats_fg] = pitcher_fg_scaler.transform(pitchers_merged[pitcher_stats_fg])
        
        # Determine clusters 
        pitcher_stats_fg = pitcher_stats_fg.append('p_L')
        pitchers_merged['Cluster'] = np.nan
        for i, row in pitchers_merged.iterrows():
            try:
                prediction = pitcher_kmeans.predict([row[['H/9', 'HR/9','K/9', 'BB/9', 'p_L']]])
                pitchers_merged.loc[i, 'Cluster'] = prediction[0]
            except:
                pitchers_merged.loc[i, 'Cluster'] = 1
        
        # Merge with clusters
        pitchers_merged = pd.merge(pitchers_merged, pitcher_clusters, on='Cluster', suffixes=("", "_cl"), how='left')
        
        # Fill in small sample 
        for stat in pitcher_stats:
            pitchers_merged[f'{stat}_l'] = np.where(pitchers_merged['pa_p_l'] < 40, pitchers_merged[f'{stat}_l_cl'], pitchers_merged[f'{stat}_l'])
            pitchers_merged[f'{stat}_r'] = np.where(pitchers_merged['pa_p_r'] < 40, pitchers_merged[f'{stat}_r_cl'], pitchers_merged[f'{stat}_r'])
            
        # Drop cluster columns
        columns_to_drop = [col for col in pitchers_merged.columns if col.endswith('_cl')]
        pitchers_merged.drop(columns=columns_to_drop, inplace=True)
            
            
        ## Normalize stats
        # Left
        # Rename stats to make compatible with scaler
        for stat in pitcher_stats:
            # Get the column name with the '_l' suffix
            column_l = stat + '_l'

            # Rename the column by removing the '_l' suffix
            pitchers_merged.rename(columns={column_l: stat}, inplace=True)

        # Apply the scaler to the renamed column
        pitchers_merged[pitcher_stats] = pitcher_scaler.transform(pitchers_merged[pitcher_stats])

        for stat in pitcher_stats:
            # Get the column name with the '_l' suffix
            column_l = stat + '_l'
            
            pitchers_merged.rename(columns={stat: column_l}, inplace=True)
            
        # Right
        for stat in pitcher_stats:
            # Get the column name with the '_r' suffix
            column_r = stat + '_r'

            # Rename the column by removing the '_r' suffix
            pitchers_merged.rename(columns={column_r: stat}, inplace=True)

        # Apply the scaler to the renamed column
        pitchers_merged[pitcher_stats] = pitcher_scaler.transform(pitchers_merged[pitcher_stats])

        for stat in pitcher_stats:
            # Get the column name with the '_r' suffix
            column_r = stat + '_r'
            
            pitchers_merged.rename(columns={stat: column_r}, inplace=True)      
        
        
        pitchers_merged.rename(columns={'H/9':'H9', 'HR/9':'HR9','K/9':'K9','BB/9':'BB9'}, inplace=True)

        
        # Save as Excel
        with pd.ExcelWriter(file_name, mode='a', engine='openpyxl') as writer:  
            pitchers_merged.to_excel(writer, sheet_name='Pitchers')

### Run One

In [17]:
# todaysdate = "20230702"
create_team_rosters(todaysdate)
# create_team_rosters("20230630")

ARI20230707.csv
ATL20230707.csv
BAL20230707.csv
BOS20230707.csv
CHC20230707.csv
CHW20230707.csv
CIN20230707.csv
CLE20230707.csv
COL20230707.csv
DET20230707.csv
HOU20230707.csv
KCR20230707.csv
LAA20230707.csv
LAD20230707.csv
MIA20230707.csv
MIL20230707.csv
MIN20230707.csv
NYM20230707.csv
NYY20230707.csv
OAK20230707.csv
PHI20230707.csv
PIT20230707.csv
SDP20230707.csv
SEA20230707.csv
SFG20230707.csv
STL20230707.csv
TBR20230707.csv
TEX20230707.csv
TOR20230707.csv
WSN20230707.csv


### Run All

In [18]:
# # Loop over the folders in the directory
# directory = r"C:\Users\james\Documents\MLB\Data2\6. Rosters"
# for folder_name in os.listdir(directory):
#     # Construct the full path of the folder
#     folder_path = os.path.join(directory, folder_name)
    
#     # Check if the path is a directory
#     if os.path.isdir(folder_path):
#         # Print the folder name
#         date = folder_name[7:15]
        
#         try:
#             create_team_rosters(date)
#         except:
#             print("Missing for {}.".format(date))

In [19]:
print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))

Code was last run on: 2023-07-07 at 16:55:57.
